Advanced implementation of MLEnd Code

In [47]:
!pip install librosa
!pip install xgboost


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 10.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [36]:
#required dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa

import zipfile
import sklearn
import random

import warnings

warnings.filterwarnings('ignore')


In [10]:
sample_path = '/home/jovyan/ML/DATA/DATA_unzip_full/*.wav'
files = glob.glob(sample_path)
len(files)

2500

In [11]:
MLENDLS_df = pd.read_csv('/home/jovyan/ML/git-code/MLEndLS_final.csv').set_index('file_id') 

In [12]:
MLENDLS_df['area'].value_counts()

campus        823
westend       666
british       404
Euston        234
southbank     229
kensington    144
Name: area, dtype: int64

In [13]:
def getPitch(x,fs,winLen=0.02):
  #winLen = 0.02 
  p = winLen*fs
  frame_length = int(2**int(p-1).bit_length())
  hop_length = frame_length//2
  f0, voiced_flag, voiced_probs = librosa.pyin(y=x, fmin=80, fmax=450, sr=fs,
                                                 frame_length=frame_length,hop_length=hop_length)
  return f0,voiced_flag

In [14]:
def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
  X,y =[],[]
  for file in tqdm(files):
    fileID = file.split('/')[-1]
    file_name = file.split('/')[-1]
    

    
    try:
        
        fs = None # if None, fs would be 22050 #fr = sr = sampling rate
        x, fs = librosa.load(file,sr=fs)

        if scale_audio: 
            #x = x/np.max(np.abs(x))
            x = sklearn.preprocessing.minmax_scale(x, axis=0)
            x = librosa.effects.preemphasis(x)

        f0, voiced_flag = getPitch(x,fs,winLen=0.02)

        #yi = labels_file.loc[fileID]['in_out']=='indoor'
        y1 = labels_file.loc[fileID]['in_out']
        y2 = labels_file.loc[fileID]['area']
        y3 = labels_file.loc[fileID]['spot']
        y4 = labels_file.loc[fileID]['Participant']
        y5 = fileID
        yi = [ y1, y2, y3, y4, y5 ]

        power = np.sum(x**2)/len(x)
        pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
        pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
        voiced_fr = np.mean(voiced_flag)

        mfccs = librosa.feature.mfcc(y = x, sr=fs)
        spectral_centroids = librosa.feature.spectral_centroid(y = x, sr=fs)[0]

        rmse = librosa.feature.rms(y=x)
        chroma_stft = librosa.feature.chroma_stft(y=x, sr=fs)
        spec_bw = librosa.feature.spectral_bandwidth(y=x, sr=fs)
        rolloff = librosa.feature.spectral_rolloff(y=x, sr=fs)
        zcr = librosa.feature.zero_crossing_rate(x)

        xi = [power,pitch_mean,pitch_std,voiced_fr, np.mean(np.mean(mfccs.T,axis=0)), np.mean(spectral_centroids), np.mean(rmse), np.mean(chroma_stft), np.mean(spec_bw), np.mean(rolloff), np.mean(zcr)]
        X.append(xi)
        y.append(yi)
        
    except:
        print('issue in file index -->' + str(file))
      
    

  return np.array(X),np.array(y)

In [6]:
#sample_file = random.sample(files, 500)

In [15]:
X,y = getXy(files, labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)

100%|██████████| 2500/2500 [2:32:25<00:00,  3.66s/it]  


In [16]:
print('The shape of X is', X.shape) 
print('The shape of y is', y.shape)

The shape of X is (2500, 11)
The shape of y is (2500, 5)


In [17]:
df1 = pd.DataFrame.from_records(X)
df1

,0,1,2,3,4,5,6,7,8,9,10
0,0.000306,0.000000,0.000000,0.000000,-9.704526,3924.455408,0.017407,0.662082,3863.928224,7658.032325,0.044540
1,0.000239,168.840522,27.394508,0.054902,-12.685133,3354.621682,0.014910,0.810797,4373.337647,8323.694853,0.009006
2,0.000415,0.000000,0.000000,0.000000,-11.732181,3855.839005,0.020163,0.571700,3295.748205,7031.123140,0.087092
3,0.000469,0.000000,0.000000,0.000000,-8.620403,5825.834241,0.021463,0.694621,4406.635488,11211.221849,0.112260
4,0.000279,237.049695,55.445517,0.113145,-15.675273,2271.428776,0.016332,0.455131,2419.640870,4578.833211,0.024275
...,...,...,...,...,...,...,...,...,...,...,...
2495,0.000318,127.043657,11.591387,0.095469,-14.222832,2130.724309,0.017640,0.639185,2652.933177,4891.556290,0.014187
2496,0.000459,89.758892,4.978481,0.004886,-16.421215,4148.489347,0.020931,0.541413,3423.110084,7694.718292,0.108947
2497,0.000226,106.654105,2.121523,0.027548,-9.625125,6846.068155,0.014362,0.698250,5644.173536,14005.658009,0.080098
2498,0.000180,0.000000,0.000000,0.000000,-13.214108,4765.394747,0.013292,0.747408,5187.487511,11065.180833,0.007154


In [18]:
df2 = pd.DataFrame.from_records(y)
df2

,0,1,2,3,4
0,outdoor,british,street,S81,1091.wav
1,outdoor,campus,square,S183,2114.wav
2,indoor,westend,market,S64,1313.wav
3,outdoor,british,street,S151,0001.wav
4,indoor,westend,charing,S94,1614.wav
...,...,...,...,...,...
2495,indoor,westend,national,S97,0804.wav
2496,indoor,campus,ground,S106,1143.wav
2497,outdoor,Euston,forecourt,S47,1733.wav
2498,outdoor,campus,square,S29,0510.wav


In [19]:
df3 = df1.join(df2, lsuffix="_x", rsuffix="_y")
df3

,0_x,1_x,2_x,3_x,4_x,5,6,7,8,9,10,0_y,1_y,2_y,3_y,4_y
0,0.000306,0.000000,0.000000,0.000000,-9.704526,3924.455408,0.017407,0.662082,3863.928224,7658.032325,0.044540,outdoor,british,street,S81,1091.wav
1,0.000239,168.840522,27.394508,0.054902,-12.685133,3354.621682,0.014910,0.810797,4373.337647,8323.694853,0.009006,outdoor,campus,square,S183,2114.wav
2,0.000415,0.000000,0.000000,0.000000,-11.732181,3855.839005,0.020163,0.571700,3295.748205,7031.123140,0.087092,indoor,westend,market,S64,1313.wav
3,0.000469,0.000000,0.000000,0.000000,-8.620403,5825.834241,0.021463,0.694621,4406.635488,11211.221849,0.112260,outdoor,british,street,S151,0001.wav
4,0.000279,237.049695,55.445517,0.113145,-15.675273,2271.428776,0.016332,0.455131,2419.640870,4578.833211,0.024275,indoor,westend,charing,S94,1614.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.000318,127.043657,11.591387,0.095469,-14.222832,2130.724309,0.017640,0.639185,2652.933177,4891.556290,0.014187,indoor,westend,national,S97,0804.wav
2496,0.000459,89.758892,4.978481,0.004886,-16.421215,4148.489347,0.020931,0.541413,3423.110084,7694.718292,0.108947,indoor,campus,ground,S106,1143.wav
2497,0.000226,106.654105,2.121523,0.027548,-9.625125,6846.068155,0.014362,0.698250,5644.173536,14005.658009,0.080098,outdoor,Euston,forecourt,S47,1733.wav
2498,0.000180,0.000000,0.000000,0.000000,-13.214108,4765.394747,0.013292,0.747408,5187.487511,11065.180833,0.007154,outdoor,campus,square,S29,0510.wav


In [20]:
df3.to_csv('feature-set-4-adv-full.csv')

In [37]:
df_adv = pd.read_csv('/home/jovyan/ML/git-code/feature-set-4-adv-full.csv').set_index('id') 
df_adv

,0_x,1_x,2_x,3_x,4_x,5,6,7,8,9,10,0_y,1_y,2_y,3_y,4_y
id,,,,,,,,,,,,,,,,
0,0.000306,0.000000,0.000000,0.000000,-9.704526,3924.455408,0.017407,0.662082,3863.928224,7658.032325,0.044540,outdoor,british,street,S81,1091.wav
1,0.000239,168.840522,27.394508,0.054902,-12.685133,3354.621682,0.014910,0.810797,4373.337647,8323.694853,0.009006,outdoor,campus,square,S183,2114.wav
2,0.000415,0.000000,0.000000,0.000000,-11.732181,3855.839005,0.020163,0.571700,3295.748205,7031.123140,0.087092,indoor,westend,market,S64,1313.wav
3,0.000469,0.000000,0.000000,0.000000,-8.620403,5825.834241,0.021463,0.694621,4406.635488,11211.221849,0.112260,outdoor,british,street,S151,0001.wav
4,0.000279,237.049695,55.445517,0.113145,-15.675273,2271.428776,0.016332,0.455131,2419.640870,4578.833211,0.024275,indoor,westend,charing,S94,1614.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,0.000318,127.043657,11.591387,0.095469,-14.222832,2130.724309,0.017640,0.639185,2652.933177,4891.556290,0.014187,indoor,westend,national,S97,0804.wav
2496,0.000459,89.758892,4.978481,0.004886,-16.421215,4148.489347,0.020931,0.541413,3423.110084,7694.718292,0.108947,indoor,campus,ground,S106,1143.wav
2497,0.000226,106.654105,2.121523,0.027548,-9.625125,6846.068155,0.014362,0.698250,5644.173536,14005.658009,0.080098,outdoor,Euston,forecourt,S47,1733.wav


In [38]:
df_adv['1_y'].value_counts()

campus        823
westend       666
british       404
Euston        234
southbank     229
kensington    144
Name: 1_y, dtype: int64

In [39]:
options = ['campus', 'westend']
df_1 = df_adv.loc[df_adv['1_y'].isin(options)]
df_1

,0_x,1_x,2_x,3_x,4_x,5,6,7,8,9,10,0_y,1_y,2_y,3_y,4_y
id,,,,,,,,,,,,,,,,
1,0.000239,168.840522,27.394508,0.054902,-12.685133,3354.621682,0.014910,0.810797,4373.337647,8323.694853,0.009006,outdoor,campus,square,S183,2114.wav
2,0.000415,0.000000,0.000000,0.000000,-11.732181,3855.839005,0.020163,0.571700,3295.748205,7031.123140,0.087092,indoor,westend,market,S64,1313.wav
4,0.000279,237.049695,55.445517,0.113145,-15.675273,2271.428776,0.016332,0.455131,2419.640870,4578.833211,0.024275,indoor,westend,charing,S94,1614.wav
6,0.000509,113.138972,0.653509,0.003145,-14.656598,3535.322484,0.021230,0.511548,2536.600340,6111.062104,0.105631,indoor,campus,ground,S146,0432.wav
9,0.000333,118.070446,2.990236,0.034146,-13.770185,1841.940028,0.016806,0.799612,2596.393599,4294.385957,0.007886,indoor,westend,national,S159,0578.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2494,0.000326,169.450828,5.888096,0.045381,-12.842073,5626.861872,0.016960,0.684174,5396.276031,11099.127267,0.033073,indoor,campus,curve,S69,1044.wav
2495,0.000318,127.043657,11.591387,0.095469,-14.222832,2130.724309,0.017640,0.639185,2652.933177,4891.556290,0.014187,indoor,westend,national,S97,0804.wav
2496,0.000459,89.758892,4.978481,0.004886,-16.421215,4148.489347,0.020931,0.541413,3423.110084,7694.718292,0.108947,indoor,campus,ground,S106,1143.wav


In [40]:
X = df_1.drop(['1_y', '2_y', '3_y', '4_y'], axis=1)
#print(X['0_y'].value_counts())

from sklearn import preprocessing

le = preprocessing.LabelEncoder()

le.fit(X['0_y'])
print(list(le.classes_))
X['0_y'] = le.transform(X['0_y'])

# le.fit(X['2_y'])
# print(list(le.classes_))
# X['2_y'] = le.transform(X['2_y'])



X

['indoor', 'outdoor']


,0_x,1_x,2_x,3_x,4_x,5,6,7,8,9,10,0_y
id,,,,,,,,,,,,
1,0.000239,168.840522,27.394508,0.054902,-12.685133,3354.621682,0.014910,0.810797,4373.337647,8323.694853,0.009006,1
2,0.000415,0.000000,0.000000,0.000000,-11.732181,3855.839005,0.020163,0.571700,3295.748205,7031.123140,0.087092,0
4,0.000279,237.049695,55.445517,0.113145,-15.675273,2271.428776,0.016332,0.455131,2419.640870,4578.833211,0.024275,0
6,0.000509,113.138972,0.653509,0.003145,-14.656598,3535.322484,0.021230,0.511548,2536.600340,6111.062104,0.105631,0
9,0.000333,118.070446,2.990236,0.034146,-13.770185,1841.940028,0.016806,0.799612,2596.393599,4294.385957,0.007886,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2494,0.000326,169.450828,5.888096,0.045381,-12.842073,5626.861872,0.016960,0.684174,5396.276031,11099.127267,0.033073,0
2495,0.000318,127.043657,11.591387,0.095469,-14.222832,2130.724309,0.017640,0.639185,2652.933177,4891.556290,0.014187,0
2496,0.000459,89.758892,4.978481,0.004886,-16.421215,4148.489347,0.020931,0.541413,3423.110084,7694.718292,0.108947,0


In [41]:
# X = df_1.drop(['1_y', '0_y', '2_y'], axis=1)
y = df_1['1_y']
y

id
1        campus
2       westend
4       westend
6        campus
9       westend
         ...   
2494     campus
2495    westend
2496     campus
2498     campus
2499    westend
Name: 1_y, Length: 1489, dtype: object

In [42]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y)

list(le.classes_)
#le.transform(['british', 'campus', 'westend'])

['campus', 'westend']

In [43]:
y_n = le.transform(y)
y_n.shape

(1489,)

In [44]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y_n,test_size=0.2)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1191, 12), (298, 12), (1191,), (298,))

In [45]:
X_train

,0_x,1_x,2_x,3_x,4_x,5,6,7,8,9,10,0_y
id,,,,,,,,,,,,
2081,0.000343,432.237644,12.873123,0.053468,-9.991570,3396.003230,0.018359,0.583604,3049.030569,7218.446235,0.044886,0
318,0.000271,0.000000,0.000000,0.000000,-13.470184,7419.260126,0.016244,0.710177,4652.064409,13147.949467,0.235327,1
2092,0.000339,0.000000,0.000000,0.000000,-8.742596,4860.002037,0.018224,0.730572,4482.466801,10257.310980,0.025627,1
1882,0.000305,204.166241,64.522717,0.153527,-13.812462,2256.749064,0.017355,0.563995,2972.338845,5364.394450,0.006723,1
1597,0.000189,0.000000,0.000000,0.000000,-13.743037,3711.748858,0.013467,0.770462,3890.042506,8378.473328,0.012956,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2360,0.000456,103.149625,0.000000,0.001531,-12.374022,6318.681684,0.021146,0.579934,5018.208208,12408.533033,0.157628,0
1231,0.000255,144.650385,61.846869,0.064993,-15.332685,2115.008424,0.015652,0.839214,2978.712229,5197.326426,0.007919,1
2253,0.000254,0.000000,0.000000,0.000000,-14.578203,5776.123300,0.014003,0.611421,5830.311715,12717.766411,0.017973,1


In [34]:
from keras import layers
from keras import layers
import keras
from keras.models import Sequential

model = Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier = model.fit(X_train, y_train, epochs = 100, batch_size = 128)

2022-12-08 22:03:26.613483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 22:03:26.775447: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-08 22:03:26.832430: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 22:03:27.441412: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

Epoch 1/100
10/10 [==============================] - 2s 2ms/step - loss: 2.0041 - accuracy: 0.4819
Epoch 2/100
10/10 [==============================] - 0s 2ms/step - loss: 1.1548 - accuracy: 0.5693
Epoch 3/100
 1/10 [==>...........................] - ETA: 0s - loss: 0.8399 - accuracy: 0.5938

2022-12-08 22:03:33.435587: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


10/10 [==============================] - 0s 2ms/step - loss: 0.6983 - accuracy: 0.6272
Epoch 4/100
10/10 [==============================] - 0s 1ms/step - loss: 0.6130 - accuracy: 0.6633
Epoch 5/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5936 - accuracy: 0.6717
Epoch 6/100
10/10 [==============================] - 0s 2ms/step - loss: 0.5847 - accuracy: 0.6499
Epoch 7/100
10/10 [==============================] - 0s 2ms/step - loss: 0.5678 - accuracy: 0.6977
Epoch 8/100
10/10 [==============================] - 0s 2ms/step - loss: 0.5574 - accuracy: 0.6944
Epoch 9/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5500 - accuracy: 0.7070
Epoch 10/100
10/10 [==============================] - 0s 2ms/step - loss: 0.5422 - accuracy: 0.7128
Epoch 11/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5358 - accuracy: 0.7305
Epoch 12/100
10/10 [==============================] - 0s 1ms/step - loss: 0.5321 - accuracy: 0.7296
Epoch 13/100
10/10 

In [35]:
test_loss, test_acc = model.evaluate(X_val,y_val)

10/10 [==============================] - 0s 1ms/step - loss: 1.1944 - accuracy: 0.6644


In [31]:
from sklearn import svm

model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)
# print(y_train)
# print(yt_p)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

Training Accuracy 0.5524769101595298
Validation  Accuracy 0.5536912751677853


In [32]:
mean = X_train.mean(0)
sd =  X_train.std(0)

X_train = (X_train-mean)/sd
X_val  = (X_val-mean)/sd

model  = svm.SVC(C=1,gamma=2)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)


print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

Training Accuracy 0.9916036943744753
Validation  Accuracy 0.62751677852349


In [33]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression(max_iter=10000)
logisticRegr.fit(X_train,y_train)

yt_p1 = model.predict(X_train)
yv_p1 = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p1==y_train))
print('Validation  Accuracy', np.mean(yv_p1==y_val))

Training Accuracy 0.9916036943744753
Validation  Accuracy 0.62751677852349


In [48]:
from xgboost import XGBClassifier

params = {
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 2.0,
            'n_estimators':100
        }

modelXGB = XGBClassifier(**params)
modelXGB.fit(X_train, y_train)


yt_p1 = modelXGB.predict(X_train)
yv_p1 = modelXGB.predict(X_val)

print('Training Accuracy', np.mean(yt_p1==y_train))
print('Validation  Accuracy', np.mean(yv_p1==y_val))

Training Accuracy 0.8253568429890848
Validation  Accuracy 0.6845637583892618
